<a href="https://colab.research.google.com/github/arj1211/ece495/blob/main/A3/w23_ece495_assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECE 493 Assignment 3

Assignment Overview
- Learn pytorch and helper libraries
- Understand VOC Sementic Segmentation Dataset
- Create data transforms to augment the dataset
- Create the neural network
- Train and evaluate the neural network

Problems:
- 1) Data augmentation
- 2) Create the neural network

Note:
- This assignment was created based on assignment 4 of CS 484
 - https://cs.uwaterloo.ca/~yboykov/Courses/cs484/

# Importing libraries

Everything that you need for this assignment is imported here.

The VoxSegmentation dataset contains images and segmented images.
https://chainercv.readthedocs.io/en/stable/reference/datasets.html#chainercv.datasets.VOCSemanticSegmentationDataset


In [ ]:
# You can run this cell to install packages
# I believe this is the only package not preinstalled on google colab
!pip install chainercv

In [ ]:
# pytorch library
import torch
# neural network layers (Conv2d, Linear, etc.) that will be trained
# https://pytorch.org/docs/stable/nn.html
import torch.nn as nn
# Many functions (convolution, pooling, activation, etc.)
# https://pytorch.org/docs/stable/nn.functional.html
import torch.nn.functional as F
# https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
from torch.utils.data import DataLoader

# Contains pretrained models for transfer learning
# https://pytorch.org/docs/stable/torchvision/models.html
import torchvision.models as models
# import data transforms
# https://pytorch.org/docs/stable/torchvision/transforms.html
# Take note of transforms.ToPILImage and transforms.ToTensor
import torchvision.transforms as transforms


# Use cuda on the gpu or use the cpu
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# Overide device when testing on CPU
# device = 'cpu'

# graphing and images
import matplotlib.pyplot as plt
from PIL import Image

# numpy
import numpy as np

# deepcopy the nn
import copy

# for random number generation
import random

# chainerCV is similar to torchvision
import chainercv
# https://chainercv.readthedocs.io/en/stable/reference/datasets.html#chainercv.datasets.VOCSemanticSegmentationDataset
from chainercv.datasets import VOCSemanticSegmentationDataset
# https://docs.chainer.org/en/stable/reference/generated/chainer.datasets.TransformDataset.html#chainer.datasets.TransformDataset
from chainer.datasets import TransformDataset

# Used to calculate mIoU
from chainercv.evaluations import eval_semantic_segmentation


# Import VOC Segmentation Dataset

In [ ]:
import os.path
from os import path
from google.colab import drive

drive.mount('/content/gdrive')
DATASET_LOCATION = '/content/gdrive/MyDrive/'

if not path.exists(DATASET_LOCATION + 'VOC2012'):
  print('First time run, downloading VOC and copying to permanent location')
  # auto will download the dataset to $HOME/.chainer/dataset
  # This gets deleted when you disconnect from google colab
  voc_train_data = VOCSemanticSegmentationDataset(data_dir='auto', split='train')
  !cp -r $HOME/.chainer/dataset/pfnet/chainercv/voc/VOCdevkit/VOC2012/ /content/gdrive/MyDrive/

# Instantiate the data loader with the dataset directory
voc_train_data = VOCSemanticSegmentationDataset(data_dir=DATASET_LOCATION + 'VOC2012', split='train')

print("Number of training examples:", len(voc_train_data))
print("Each with one image and one ground truth (gt) segmented image:", len(voc_train_data[0]))

print("Image Shape:", voc_train_data[0][0].shape)
print("Segmented Image Shape:", voc_train_data[0][1].shape)

Mounted at /content/gdrive
Number of training examples: 1464


There are 22 Classes in VOC.

https://github.com/NVIDIA/DIGITS/blob/master/examples/semantic-segmentation/pascal-voc-classes.txt
- -1 is ignore
- 0 is background
- 1-20 are different objects

In [ ]:
# Find images with people and cars
def find_car_and_ppl_images():
    car_label = 7
    person_label = 15
    count = 0
    max_count = 100
    for i in range(len(voc_train_data)):
        tmp_count = np.unique(voc_train_data[i][1], return_counts=True)
        if car_label in tmp_count[0] and person_label in tmp_count[0]:
            if count == max_count:
                break
            else:
                img, label = voc_train_data[i]
                fig = plt.figure(figsize=(4,3))
                ax = fig.add_subplot(1,1,1)
                plt.title('Image ' + str(i))
                ax.imshow(np.rollaxis(img.astype(int), 0, 3))
                count += 1

# Some relevant image indexes for a self-driving car
# Car = 119, 273, Bicycle = 225
img_119, seg_img_119 = voc_train_data[119]
img_273, seg_img_273 = voc_train_data[273]
img_225, seg_img_225 = voc_train_data[225]

print("View count of pixels within each label category:")
print(np.unique(voc_train_data[119][1], return_counts=True))
print(np.unique(voc_train_data[273][1], return_counts=True))
print(np.unique(voc_train_data[225][1], return_counts=True))

print("We must move the RGB channel to the back to be viewed with plt:")
print("Before:", img_119.shape)
print("After:", np.rollaxis(img_119, 0, 3).shape)

print("Lastly let's verify that the image and gt label match")

# This colorize_mask class takes in a numpy segmentation mask,
#  and then converts it to a PIL Image for visualization.
#  Since by default the numpy matrix contains integers from
#  0,1,...,num_classes, we need to apply some color to this
#  so we can visualize easier! Refer to:
#  https://pillow.readthedocs.io/en/4.1.x/reference/Image.html#PIL.Image.Image.putpalette
palette = [0, 0, 0, 128, 0, 0, 0, 128, 0, 128, 128, 0, 0, 0, 128, 128, 0, 128, 0, 128, 128,
           128, 128, 128, 64, 0, 0, 192, 0, 0, 64, 128, 0, 192, 128, 0, 64, 0, 128, 192, 0, 128,
           64, 128, 128, 192, 128, 128, 0, 64, 0, 128, 64, 0, 0, 192, 0, 128, 192, 0, 0, 64, 128]

def colorize_mask(mask):
    new_mask = Image.fromarray(mask.astype(np.uint8)).convert('P')
    new_mask.putpalette(palette)

    return new_mask

def add_img_plot(fig, index, img, title, sub_plot_id):
    ax = fig.add_subplot(sub_plot_id[0],sub_plot_id[1],sub_plot_id[2])
    plt.title(title + str(index))
    ax.imshow(img)

# Create the figure size
fig = plt.figure(figsize=(10,10))
# Car image 119
add_img_plot(fig, 119, np.rollaxis(img_119.astype(int), 0, 3), 'Image ', [3,2,1])
add_img_plot(fig, 119, colorize_mask(seg_img_119), 'Segmented Image ', [3,2,2])
# Car image 273
add_img_plot(fig, 273, np.rollaxis(img_273.astype(int), 0, 3), 'Image ', [3,2,3])
add_img_plot(fig, 273, colorize_mask(seg_img_273), 'Segmented Image ', [3,2,4])
# Bicycle image 225
add_img_plot(fig, 225, np.rollaxis(img_225.astype(int), 0, 3), 'Image ', [3,2,5])
add_img_plot(fig, 225, colorize_mask(seg_img_225), 'Segmented Image ', [3,2,6])

# Problem 1 Data Augmentation

The transform below is used to create a transformed training dataset. It currently center crops all images.

Modify it so that:
- There is a 50% chance the images will be flipped horizontally
- Instead of a center crop, a random part of the image is cropped

Useful functions:
- chainercv.transforms.flip
- chainercv.transforms.random_crop

Note: The output image and segmented images should match

In [ ]:
# From ImageNet dataset
norm = ([0.485, 0.456, 0.406], 
        [0.229, 0.224, 0.225])

def tensor_to_img(tensor):
    # Denormalize tensor and convert to uint8 np array for visualization
    denormalize = transforms.Normalize(
        mean=[-m / s for m, s in zip(norm[0], norm[1])],
        std=[1.0 / s for s in norm[1]]
    )
    np_img = np.moveaxis((denormalize(tensor).numpy() * 255), -3, -1).astype(np.uint8)
    
    return np_img

# Modify this function
def voc_train_transform(in_data):
    size = (110,110)
    img, seg_img = in_data

    # Add channel to seg img
    seg_img = seg_img[None]
    
    # # Center crop
    # img = chainercv.transforms.center_crop(img, size)
    # seg_img = chainercv.transforms.center_crop(seg_img, size)[0]
    
    # 50% random horiz flip
    if np.random.random() < 0.5:
        img = chainercv.transforms.flip(img, x_flip=True)
        seg_img = chainercv.transforms.flip(seg_img, x_flip=True)
    
    # Random crop
    img, crop_param = chainercv.transforms.random_crop(img, size, return_param=True)
    seg_img = seg_img[:, crop_param['y_slice'], crop_param['x_slice']][0]

    # Convert input img to tensor and normalize
    img = np.rollaxis(img, 0, 3)
    img = transforms.ToPILImage()(np.uint8(img))
    img = transforms.ToTensor()(img)
    img = transforms.Normalize(norm[0],norm[1])(img)

    return img, torch.from_numpy(seg_img.copy()).long()

img_119, seg_img_119 = voc_train_transform(voc_train_data[119])
img_273, seg_img_273 = voc_train_transform(voc_train_data[273])
img_225, seg_img_225 = voc_train_transform(voc_train_data[225])

# Create the figure size
fig = plt.figure(figsize=(10,10))
# Car image 119
add_img_plot(fig, 119, tensor_to_img(img_119), 'Image ', [3,2,1])
add_img_plot(fig, 119, colorize_mask(seg_img_119.numpy()), 'Segmented Image ', [3,2,2])
# Car image 273
add_img_plot(fig, 273, tensor_to_img(img_273), 'Image ', [3,2,3])
add_img_plot(fig, 273, colorize_mask(seg_img_273.numpy()), 'Segmented Image ', [3,2,4])
# Bicycle image 225
add_img_plot(fig, 225, tensor_to_img(img_225), 'Image ', [3,2,5])
add_img_plot(fig, 225, colorize_mask(seg_img_225.numpy()), 'Segmented Image ', [3,2,6])

# Dataloader

In [ ]:
voc_train_data_transformed = TransformDataset(voc_train_data, voc_train_transform)

# epoch = one iteration over the entire dataset
# batch size = samples per batch within an epoch
# shuffle = Set to True if your data is ordered
# num_workers = number of subprocess to use when dataloading, set higher if you have many cpu cores
if device == 'cpu':
    train_loader = DataLoader(voc_train_data_transformed, batch_size=5, shuffle=True, num_workers=10)
else:
    train_loader = DataLoader(voc_train_data_transformed, batch_size=50, shuffle=True, num_workers=2)

# Problem 2 Create the neural network

In [ ]:
t = (3, 4)
t = tuple(i+2 for i in t)
t

In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes, criterion=None):
        super(Net, self).__init__()
        self.num_classes = num_classes
        self.criterion = criterion
        # Hints:
        # Useful layers:
        # - nn.Conv2d
        # - nn.BatchNorm2d
        # - nn.Dropout2d

        # Import a pre-trained encoder
        self.resnet = models.resnet34(pretrained=True)

        # Decoder layers
        self.conv1 = nn.Conv2d(in_channels=640, out_channels=64, kernel_size=(5,5))
        self.bn1 = nn.BatchNorm2d(64)
        self.do1 = nn.Dropout2d(0.45)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=21, kernel_size=(3,3))


    def forward(self, inp):
        # Fill in model to produce lfinal
        inp_dims = inp.size(2), inp.size(3)
        print('inp', inp.shape)
        
        # Hints: Useful functions
        # - F.relu        (neuron activation)
        # - F.interpolate (interpolate to increase tensor H and W)
        # - torch.cat     (concatenate tensors to implement a skip connection)

        # Layer 1
        x0 = self.resnet.conv1(inp)
        print('x0', x0.shape)
        x0 = self.resnet.bn1(x0)
        print('x0', x0.shape)
        x0 = self.resnet.relu(x0)
        print('x0', x0.shape)
        x0 = self.resnet.maxpool(x0)
        print('x0', x0.shape)

        # Layer 2
        x1 = self.resnet.layer1(x0)
        print('x1', x1.shape)

        # Layer 3
        x2 = self.resnet.layer2(x1)
        print('x2', x2.shape)
        x2_dims = x2.size(2), x2.size(3)
        
        # Layer 4
        x3 = self.resnet.layer3(x2)
        print('x3', x3.shape)
        
        # Layer 5
        x4 = self.resnet.layer4(x3)
        print('x4', x4.shape)

        c_inp_1 = F.interpolate(x4, size=x2_dims)
        print('c_inp_1', c_inp_1.shape)

        c_out = torch.cat((c_inp_1, x2), 1)
        print('c_out', c_out.shape)


        y1 = self.conv1(c_out)
        print('y1', y1.shape)
        y1 = self.bn1(y1)
        print('y1', y1.shape)
        y1 = F.relu(y1)
        print('y1', y1.shape)
        y1 = self.do1(y1)
        print('y1', y1.shape)

        inp_dims = tuple(i+2 for i in inp_dims)
        y2 = F.interpolate(y1, size=inp_dims)
        print('y2', y2.shape)

        y3 = self.conv2(y2)
        print('y3', y3.shape)

        # if self.training:
        #     return self.criterion(lfinal, gts)
        # else:
        #     return lfinal

        # Return the final prediction
        lfinal = y3
        return lfinal


In [ ]:
untrained_net = Net(21).to(device)
untrained_net.eval()

sample_img, sample_target = voc_train_data_transformed[119]

untrained_output = untrained_net.forward(sample_img[None].to(device))
if device != 'cpu':
    untrained_output = untrained_output.cpu()
untrained_nn_seg_img_119 = torch.argmax(untrained_output.cpu(), dim=1).numpy()[0]

# Create the figure size
fig = plt.figure(figsize=(8,2))
# Car image 119
add_img_plot(fig, 119, transforms.ToPILImage()(img_119), 'Image ', [1,3,1])
add_img_plot(fig, 119, colorize_mask(seg_img_119.numpy()), 'GT Segmented Image ', [1,3,2])
add_img_plot(fig, 119, colorize_mask(untrained_nn_seg_img_119), 'NN Segmented Image', [1,3,3])

In [ ]:
def train(train_loader, net, criterion, optimizer, loss_graph, device):

    for batch_idx, (data, target) in enumerate(train_loader):
        img, gt_seg_img = data.to(device), target.to(device)

        # This is a forward pass which also returns loss due to training mode
        output = net(img)
        loss = criterion(output, gt_seg_img)

        # Populate this list to graph the loss
        loss_graph.append(loss.item())
        
        # View the loss within the epoch
        print(loss.item())

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return loss

In [ ]:
%%time
%matplotlib inline

# Deep copy in order to train the network with the same initialized weights
trained_net = copy.deepcopy(untrained_net)
trained_net = trained_net.to(device)

# set loss function for the net
criterion = nn.CrossEntropyLoss(ignore_index=-1)

# You can change the number of EPOCHS
EPOCH = 40

# switch to train mode (original untrained_net was set to eval mode)
trained_net.train()

# You may switch the optimizer and tune the hyperparmeters
optimizer = torch.optim.SGD(trained_net.parameters(),
                            lr=0.001,
                            weight_decay=1e-5,
                            momentum=0.9,
                            nesterov=False)

print("Starting Training...")

loss_graph = []

fig = plt.figure(figsize=(12,6))
plt.subplots_adjust(bottom=0.2,right=0.85,top=0.95)
ax = fig.add_subplot(1,1,1)

for e in range(EPOCH):
    loss = train(train_loader, trained_net, criterion, optimizer, loss_graph, device)
    ax.clear()
    ax.set_xlabel('iterations')
    ax.set_ylabel('loss value')
    ax.set_title('Training loss curve for trained net')
    ax.plot(loss_graph, label='training loss')
    ax.legend(loc='upper right')
    fig.canvas.draw()
    print("Epoch: {} Loss: {}".format(e, loss))


In [ ]:
def validate(val_loader, net):
    iou_arr = []

    net.eval()

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            img, gt_seg_img = data.to(device), target.to(device)

            output = net(img)

            # Convert to numpy arrays
            if device != 'cpu':
                output = output.cpu()
                gt_seg_img = gt_seg_img.cpu()
            pred = torch.argmax(output, dim=1).numpy()[0]

            gt_np = gt_seg_img.numpy()[0]

            conf = eval_semantic_segmentation(pred[None], gt_np[None])

            iou_arr.append(conf['miou'])
    
    return sum(iou_arr) / len(iou_arr)

# Validation

In [ ]:
# a validation loader is created with a transform that doesn't modify the data
voc_val_data = VOCSemanticSegmentationDataset(data_dir=DATASET_LOCATION + 'VOC2012', split='val')

def val_transform(in_data):
    img, seg_img = in_data

    img = np.rollaxis(img, 0, 3)
    img = transforms.ToPILImage()(np.uint8(img))
    img = transforms.ToTensor()(img)
    img = transforms.Normalize(mean=norm[0],std=norm[1])(img)

    return img, torch.from_numpy(seg_img.copy()).long()

voc_val_data_transformed = TransformDataset(voc_val_data, val_transform)

# epoch = one iteration over the entire dataset
# batch size = samples per batch within an epoch
# shuffle = Set to True if your data is ordered
# num_workers = number of subprocess to use when dataloading, set higher if you have many cpu cores
val_loader = DataLoader(voc_val_data_transformed, batch_size=1, shuffle=True, num_workers=10)

In [ ]:
%%time
print("mIoU over the training dataset:{}".format(validate(train_loader, trained_net)))

In [ ]:
%%time
print("mIoU over the validation dataset:{}".format(validate(val_loader, trained_net)))

In [ ]:
# switch back to evaluation mode
trained_net.eval()

def add_img_txt_plot(fig, index, img, title, sub_plot_id, txt):
    ax = fig.add_subplot(sub_plot_id[0],sub_plot_id[1],sub_plot_id[2])
    plt.title(title + str(index))
    ax.text(10, 25, 'mIoU = {:_>8.6f}'.format(txt), fontsize=20, color='white')
    ax.imshow(img)

def get_nn_seg_img(net, data):
    img, gt_seg_img = data

    nn_seg_output = net.forward(img[None].cuda())

    # computing mIOU (quantitative measure of accuracy for network predictions)
    if device != 'cpu':
        nn_seg_img = torch.argmax(nn_seg_output, dim=1).cpu().numpy()[0]
    else:
        nn_seg_img = torch.argmax(nn_seg_output, dim=1).numpy()[0]

    gts = gt_seg_img.cpu().numpy()

    conf = eval_semantic_segmentation(nn_seg_img[None], gts[None])

    print("View count of pixels within each label category:")
    print(np.unique(gt_seg_img, return_counts=True))
    print(np.unique(nn_seg_img, return_counts=True))

    return nn_seg_img, conf['miou']

# Some relevant image indexes for a self-driving car
# Car = 119, 273, Bicycle = 225
img_119, seg_img_119 = voc_train_data[119]
img_273, seg_img_273 = voc_train_data[273]
img_225, seg_img_225 = voc_train_data[225]

# Get NN Output
nn_seg_img_119, miou_119 = get_nn_seg_img(trained_net, val_transform(voc_train_data[119]))
nn_seg_img_273, miou_273 = get_nn_seg_img(trained_net, val_transform(voc_train_data[273]))
nn_seg_img_225, miou_225 = get_nn_seg_img(trained_net, val_transform(voc_train_data[225]))

# Create the figure size
fig = plt.figure(figsize=(20,15))
# Car image 119
add_img_plot(fig, 119, np.rollaxis(img_119.astype(int), 0, 3), 'Image ', [3,3,1])
add_img_plot(fig, 119, colorize_mask(seg_img_119), 'GT Segmented Image ', [3,3,2])
add_img_txt_plot(fig, 119, colorize_mask(nn_seg_img_119), 'NN Segmented Image ', [3,3,3], miou_119)
# Car image 273
add_img_plot(fig, 273, np.rollaxis(img_273.astype(int), 0, 3), 'Image ', [3,3,4])
add_img_plot(fig, 273, colorize_mask(seg_img_273), 'Segmented Image ', [3,3,5])
add_img_txt_plot(fig, 273, colorize_mask(nn_seg_img_273), 'NN Segmented Image ', [3,3,6], miou_273)
# Bicycle image 225
add_img_plot(fig, 225, np.rollaxis(img_225.astype(int), 0, 3), 'Image ', [3,3,7])
add_img_plot(fig, 225, colorize_mask(seg_img_225), 'Segmented Image ', [3,3,8])
add_img_txt_plot(fig, 225, colorize_mask(nn_seg_img_225), 'NN Segmented Image ', [3,3,9], miou_225)
